## AST Project 2
### Collaborators: Arian Andalib, Ashley Stone, Jonathan Kho, Emma Oswald
### Michigan State University
### AST 304

In [1]:
########################################################################
# Team Spectacular Stellars: Arian Andalib, Ashley Stone, Jonathan Kho, Emma Oswald
# AST 304, Fall 2022
# Michigan State University
########################################################################
    
# The libraries used
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.constants as sc
%matplotlib inline
from eos import *
from astro_const import *
from observations import *
from structure import *
from test_eos import *
from scipy.optimize import bisect
import pandas as pd

In [2]:
# 2.2
!python test_eos.py

Comparing EOS to eos_table.txt...

   rho (table)    rho (test)    difference       P (table)      P (test)    difference

  1.000000e+05  1.000000e+05  5.820766e-16    6.810445e+14  6.810445e+14  0.000000e+00
  1.123655e+05  1.123655e+05  1.295052e-16    8.271108e+14  8.271108e+14  7.556424e-16
  1.262600e+05  1.262600e+05  9.220284e-16    1.004504e+15  1.004504e+15 -7.466368e-16
  1.418727e+05  1.418727e+05  4.102810e-16    1.219944e+15  1.219944e+15  2.049274e-16
  1.594159e+05  1.594159e+05  1.825654e-16    1.481591e+15  1.481591e+15  6.749502e-16
  1.791284e+05  1.791284e+05  6.498986e-16    1.799353e+15  1.799353e+15 -1.389388e-16
  2.012785e+05  2.012785e+05  7.229740e-16    2.185268e+15  2.185268e+15 -3.432074e-16
  2.261676e+05  2.261676e+05  5.147303e-16    2.653950e+15  2.653950e+15  1.883984e-16
  2.541343e+05  2.541343e+05  5.726073e-16    3.223153e+15  3.223153e+15  0.000000e+00
  2.855592e+05  2.855592e+05  8.153497e-16    3.914436e+15  3.914436e+15 -3.831970e-16
  3.208

In [3]:
def f(Pc,Mwant,delta_m,eta,xi,mue):
    m,r,p = integrate(Pc,delta_m,eta,xi,mue)
    M = m[-1]
    return(M-Mwant)

In [4]:
# 2.6 Testing integration
# Here, we tested delta_m, eta, and xi to ensure that our values for m, r, and p didn't
# change when these parameters were slightly adjusted

M = Msun
mue = 2

Pguess = pressure_guess(M,mue)

delta_m = 1e-4
eta = 1e-11
xi = 0.01

m,r,p = integrate(Pguess,delta_m,eta,xi,mue)

- We started with certain initial states for delta_m, eta, and xi. We then integrated with that function and kept reducing the delta_m value untill no noticle change in the value. After, we did the same process for eta and xi untill convergence. 

In [5]:
# masses = Msun*np.arange(0.1,1.1,0.1)
# Pcs = np.zeros(10)
# Pguesses = np.zeros(10)

# for i in range(len(masses)):
#     Pguess = pressure_guess(masses[i],mue)
#     Pguesses[i] = Pguess
#     Pc = bisect(f,Pguess/100,Pguess,args = (masses[i],delta_m,eta,xi,mue))
#     Pcs[i] = Pc

In [6]:
# For 2.6 calibration: off by this factor
# Pguesses/Pcs 

The above code was used to find the factor by which our Pguess was off from the true value of the central pressure. This gave us the array array([40.09269443, 40.09270145, 40.0926923 , 40.09269571, 40.09270147, 40.09269932, 40.09269657, 40.09270274, 40.09270293, 40.09269572]), so we introduced a factor of 40.09269 to our pressure_guess function.

In [7]:
masses = Msun*np.arange(0.1,1.1,0.1)
Pcs = np.zeros(10)
Pguesses = np.zeros(10)

for i in range(len(masses)):
    Pguess = pressure_guess(masses[i],mue)
    Pguesses[i] = Pguess
    Pc = bisect(f,Pguess/10,Pguess*10,args = (masses[i],delta_m,eta,xi,mue))
    Pcs[i] = Pc

In [8]:
Pguesses/Pcs 

array([1.00003213, 1.00003213, 1.00003213, 1.00003213, 1.00003213,
       1.00003213, 1.00003213, 1.00003213, 1.00003213, 1.00003213])

In [9]:
#2.7
central_dens = density(Pcs,2)
central_dens

array([4.06307651e+07, 1.62523061e+08, 3.65676886e+08, 6.50092242e+08,
       1.01576913e+09, 1.46270754e+09, 1.99090749e+09, 2.60036897e+09,
       3.29109198e+09, 4.06307651e+09])

In [10]:
Radii = np.zeros(10)

for i in range(len(masses)):
    m,r,p = integrate(Pcs[i],delta_m,eta,xi,mue)
    Radii[i] = r[-1]

In [11]:
R = Radii/Rsun
Pcsnorm = Pcs/(G*(masses**2)*Radii**(-4))
central_dens_norm = central_dens/((3*masses)/(4*np.pi*Radii**(3)))

In [12]:
array = np.array([masses/Msun,R,Pcs,Pcsnorm,central_dens,central_dens_norm])
MRdf = pd.DataFrame(array,index = ["$\dfrac{M}{M_{\odot}}$","$\dfrac{R}{R_\odot}$",
                                   "$P_c$","$\dfrac{P_c}{GM^2R^{-4}}$","$\rho_c$",
                                   "$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$"]).transpose()
MRdf.columns = ["$$\dfrac{M}{M_{\odot}}$$","$$\dfrac{R}{R_\odot}$$",
                                   "$$P_c$$","$$\dfrac{P_c}{GM^2R^{-4}}$$","$$\rho_c$$",
                                   "$$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$$"]
MRdf

,$$\dfrac{M}{M_{\odot}}$$,$$\dfrac{R}{R_\odot}$$,$$P_c$$,$$\dfrac{P_c}{GM^2R^{-4}}$$,$$\rho_c$$,$$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$$
0,0.1,0.027494,1.517984e+19,0.769975,4.063077e+07,5.989740
1,0.2,0.021822,1.530032e+20,0.769975,1.625231e+08,5.989738
2,0.3,0.019063,5.911141e+20,0.769975,3.656769e+08,5.989740
3,0.4,0.017320,1.542176e+21,0.769975,6.500922e+08,5.989741
4,0.5,0.016078,3.244645e+21,0.769975,1.015769e+09,5.989738
5,0.6,0.015130,5.958057e+21,0.769975,1.462708e+09,5.989738
6,0.7,0.014373,9.960030e+21,0.769975,1.990907e+09,5.989741
7,0.8,0.013747,1.554416e+22,0.769975,2.600369e+09,5.989739
8,0.9,0.013218,2.301842e+22,0.769975,3.291092e+09,5.989739
9,1.0,0.012761,3.270397e+22,0.769975,4.063077e+09,5.989741


### Questions?

- One way to do this is to adjust the step h so that it is always less than the scale over which the radius or pressure changes appreciably. What is this scale?
- This last quantity is a check on our virial relation (14): is it the same for all masses? is it unusually large or small?


- centeral pressure is about 0.77 $\frac{GM^2}{R^4}$ and central density is around 5.99 $\frac{3M}{4\pi R^2}$

In [13]:
obs.mass_errors

NameError: name 'obs' is not defined

In [ ]:
obs.radius_errors

In [ ]:
obs = MassRadiusObservations()
plt.figure(figsize = (15,10))
plt.plot(MRdf['$$\dfrac{M}{M_{\odot}}$$'],MRdf[ '$$\dfrac{R}{R_\odot}$$'],label = "Calculated",lw = 3)
plt.scatter(obs.masses,obs.radii/100,label = "Observed",color = "Red",s = 50)
plt.grid()
plt.xlabel("$M/M_\odot$",size = 25)
plt.ylabel("$R/R_\odot$",size = 25)
plt.title("Mass vs. Radius for degenerate white dwarf stars",size = 25)
plt.errorbar(obs.masses,obs.radii/100,0,obs.mass_errors,fmt=",",ecolor="black",alpha=0.7,label="mass error")
plt.errorbar(obs.masses,obs.radii/100,obs.radius_errors/100,0,fmt=",",ecolor="red",alpha=0.7,label="radius error")
plt.legend()
plt.show()

- Realism:
    Our model is not exact but it closely resembles the actual observed data and certain points are within the actual values.